In [16]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from urllib.parse import urlparse
import json

In [17]:
URL = "https://vercel.com"

In [18]:
options = webdriver.ChromeOptions()
options.add_experimental_option("useAutomationExtension", False)
options.add_experimental_option("excludeSwitches", ["enable-automation"])
driver = webdriver.Chrome(options=options)
driver.get(URL)
driver.maximize_window()

In [19]:
target_tags = ["h1", "nav", "content", "main", "h2", "h3"]
tag_elements_dictionary = {}

In [20]:
with open("./js/boundingBoxes.js", "r") as file:
    js_code = file.read()

driver.execute_script(js_code)

In [21]:
def paintCornerLabels(tag_name):
    corners = [(0, 0), (0, 1), (1, 0), (1, 1)]
    tag_elements = driver.find_elements(By.TAG_NAME, tag_name)
    for element in tag_elements:
        driver.execute_script(
            "arguments[0].style.border = 'tomato solid 2px';", element
        )
        size = element.size
        if size["width"] == 0 or size["height"] == 0:
            continue
        location = element.location
        for corner in corners:
            posX = location["x"] + (size["width"] * corner[0])
            posY = location["y"] + (size["height"] * corner[1])
            driver.execute_script(f"paintCornerLabels({posX},{posY})")

In [22]:
for tag in target_tags:
    paintCornerLabels(tag)

In [23]:
def get_tag_bounding_boxes(tag_name):
    tag_elements = driver.find_elements(By.TAG_NAME, tag_name)
    elements_bounding_boxes = []
    for element in tag_elements:
        size = element.size
        if size["width"] == 0 or size["height"] == 0:
            continue
        location = element.location
        elements_bounding_boxes.append(
            [location["x"], location["y"], size["width"], size["height"]]
        )

    return elements_bounding_boxes

In [24]:
get_tag_bounding_boxes("h1")

[[360, 291, 802, 99]]

In [25]:
bounding_boxes_dict = {}
for tag in target_tags:
    bounding_boxes_dict[tag] = get_tag_bounding_boxes(tag)

In [26]:
bounding_boxes_dict

{'h1': [[360, 291, 802, 99]],
 'nav': [[189, 15, 442, 33], [60, 7700, 1400, 197]],
 'content': [],
 'main': [[220, 155, 1080, 723]],
 'h2': [[292, 1879, 172, 23],
  [832, 1879, 216, 23],
  [292, 2461, 172, 23],
  [292, 2959, 132, 23],
  [832, 2959, 103, 23],
  [446, 3496, 117, 31],
  [709, 3496, 242, 31],
  [401, 3738, 355, 31],
  [268, 5007, 522, 31],
  [987, 5007, 132, 31],
  [268, 5379, 263, 70],
  [268, 5824, 475, 51],
  [268, 6657, 263, 147],
  [309, 7713, 319, 21],
  [652, 7713, 319, 21],
  [995, 7713, 319, 21],
  [1338, 7713, 120, 21]],
 'h3': [[292, 5972, 133, 23],
  [652, 5972, 146, 23],
  [292, 6212, 150, 23],
  [652, 6212, 155, 23],
  [1011, 6212, 144, 23]]}

In [27]:
def get_site_name(url):
    parsed_url = urlparse(url)
    domain = parsed_url.netloc
    domain_name = domain.split(".")[0]
    return domain_name

In [28]:
driver.fullscreen_window()
driver.save_screenshot(f"./dataset/images/{get_site_name(URL)}.png")

True

In [29]:
annotation = [{f"{get_site_name(URL)}.png": bounding_boxes_dict}]

In [30]:
with open("./dataset/annotations/annotations.json", "w") as json_file:
    json.dump(annotation, json_file)

In [31]:
driver.quit()